In [10]:
import numpy as np
import pandas as pd
import datetime
import time
from skimage import io, transform
from sklearn.metrics import confusion_matrix
import warnings
from sklearn.model_selection import train_test_split
from simple_models_0911 import deep, deep_cnn, elvinouyang
import keras
import numpy as np
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping
import os
import numpy as np
import pandas as pd
import random
    
from glob import glob

from scipy.io import wavfile
from scipy.signal import stft

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.models import load_model


warnings.filterwarnings("ignore")
map_file = '../train/train_map.csv'
map_df=pd.read_csv(map_file, index_col=0)

print(">>> Loading the data mapping files...")
map_df = pd.read_csv(map_file, index_col=0)
targets_to_keep = ['yes', 'no', 'up', 'down', 'left',
                   'right', 'on', 'off', 'stop', 'go', 'silence']
map_df['target'] = map_df['label']
map_df['target'] = map_df['target'].apply(
    lambda x: x if x in targets_to_keep else 'unknown')
label_to_ix = {
    'unknown': 0,
    'silence': 1,
    'down': 2,
    'go': 3,
    'left': 4,
    'no': 5,
    'off': 6,
    'on': 7,
    'right': 8,
    'stop': 9,
    'up': 10,
    'yes': 11}
map_df['label'] = map_df['target'].apply(lambda x: label_to_ix[x])

random_state=2018
test_size=0.3
val_size=0.2

INPUT_SHAPE = (99,161,3)
BATCH = 32
EPOCHS = 15

NUM_CLASSES=len(label_to_ix)

class_weight_imb = {0: 0.057,
                1: 1,
                2: 1,
                3: 1,
                4: 1,
                5: 1,
                6: 1,
                7: 1,
                8: 1,
                9: 1,
                10: 1,
                11: 1}

>>> Loading the data mapping files...


In [2]:
sample_map_df_train,sample_map_df_test=train_test_split(map_df,test_size=test_size,random_state=random_state)
sample_map_df_train,sample_map_df_val=train_test_split(sample_map_df_train,test_size=val_size,random_state=random_state)

In [20]:
def generator(df,batch_size,mode):
    print("chum")
    df_train,df_test=train_test_split(df,test_size=test_size,random_state=random_state)
    df_train,df_val= train_test_split(df_train,test_size=val_size,random_state=random_state)
    while True:
        # Depending on mode select DataFrame with paths
        if mode == 'train':
            ids = random.sample(range(df_train.shape[0]), df_train.shape[0])
        elif mode == 'val':
            ids = list(range(df_val.shape[0]))
        elif mode == 'test':
            ids = list(range(df_test.shape[0]))
            print(len(ids))
        else:
            raise ValueError('The mode should be either train, val or test.')
                
            # Create batches (for training data the batches are randomly permuted)
        for start in range(0, len(ids), batch_size):
            X_batch = []
            if mode != 'test': 
                y_batch = []
            end = min(start + batch_size, len(ids))
            i_batch = ids[start:end]
            for i in i_batch:
                pict_path=df.iloc[i,2]
                X_batch.append(io.imread(pict_path)[:,:,:3])
                if mode != 'test':
                    y_batch.append(df.iloc[i,1])
            X_batch = np.array(X_batch)

            if mode != 'test':
                y_batch = to_categorical(y_batch, num_classes = len(label_to_ix))
                yield (X_batch, y_batch)
            else:
                print("test")
                yield X_batch
                
def training_model(model_name="deep_cnn", optimizer="Adam", imbalanced=False, EPOCHS=30, learning_rate=0.005,
                   momentum=0.95,BATCH=250):
    """
     Keep adding new models first in the python script and then add them below
    """ 
    
    if model_name=="deep_cnn":
        model = deep_cnn(INPUT_SHAPE, NUM_CLASSES)
    elif model_name=="light_cnn":
        model = light_cnn(INPUT_SHAPE, NUM_CLASSES)
    elif model_name== "elvinouyang":
        model = elvinouyang(INPUT_SHAPE, NUM_CLASSES)
        
    if optimizer== 'Adam':    
        model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])
    elif optimizer=='SGD':
        model.compile(optimizer='sgd',loss='categorical_crossentropy')
    callbacks = [EarlyStopping(monitor='val_acc', patience=4, verbose=1, mode='max')]
    if imbalanced==False:
        history = model.fit_generator(generator=generator(map_df,BATCH, mode='train'),
                              steps_per_epoch=int(np.ceil(len(sample_map_df_train)/BATCH)),
                              epochs=EPOCHS,
                              verbose=1,
                              callbacks=callbacks,
                              validation_data=generator(map_df,BATCH, mode='val'),
                              validation_steps=int(np.ceil(len(sample_map_df_val)/BATCH)))
    else:
        history = model.fit_generator(generator=generator(map_df,BATCH, mode='train'),
                              steps_per_epoch=int(np.ceil(len(sample_map_df_train)/BATCH)),
                              epochs=EPOCHS,
                              verbose=1,
                              callbacks=callbacks,
                              class_weight=class_weight_imb,
                              validation_data=generator(map_df,BATCH, mode='val'),
                              validation_steps=int(np.ceil(len(sample_map_df_val)/BATCH)))
    return model
#model.save('0912_simple_cnn_model.h5')
#simple_cnn_model = load_model('0912_simple_cnn_model.h5')

def predictions(model):
    y_pred_proba = model.predict_generator(generator(map_df,BATCH, mode='test'),int(np.ceil(len(sample_map_df_test)/BATCH)),verbose=1)
    y_pred = np.argmax(y_pred_proba, axis=1)
    y_true = sample_map_df_test['label'].values
    acc_score = accuracy_score(y_true, y_pred)
    #confusion_matrix=confusion_matrix(y_true,y_pred)
    return y_pred_proba, y_pred, y_true, acc_score

In [7]:

#model.save('0912_simple_cnn_model.h5')
#simple_cnn_model = load_model('0912_simple_cnn_model.h5')

W0914 03:31:51.689017 139739584255744 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0914 03:31:51.710040 139739584255744 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0914 03:31:51.725755 139739584255744 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0914 03:31:51.740562 139739584255744 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 99, 161, 3)        0         
_________________________________________________________________
block1_conv (Conv2D)         (None, 99, 161, 32)       896       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 81, 32)        0         
_________________________________________________________________
block1_norm (BatchNormalizat (None, 50, 81, 32)        128       
_________________________________________________________________
block2_conv (Conv2D)         (None, 50, 81, 32)        9248      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 41, 32)        0         
_________________________________________________________________
block2_norm (BatchNormalizat (None, 25, 41, 32)        128       
__________

In [8]:
model_deep_cnn=training_model(model_name="deep_cnn", class_weight=False, EPOCHS=10)

W0914 18:58:46.327999 140061980559104 deprecation.py:323] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 99, 161, 3)        0         
_________________________________________________________________
block1_conv (Conv2D)         (None, 99, 161, 32)       896       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 81, 32)        0         
_________________________________________________________________
block1_norm (BatchNormalizat (None, 50, 81, 32)        128       
_________________________________________________________________
block2_conv (Conv2D)         (None, 50, 81, 32)        9248      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 41, 32)        0         
_________________________________________________________________
block2_norm (BatchNormalizat (None, 25, 41, 32)        128       
__________

In [18]:
y_pred_proba,y_pred, y_true, acc_score=predictions(model_deep_cnn)

chum
20137
test
  1/630 [..............................] - ETA: 47stest
test
  3/630 [..............................] - ETA: 31stest
test
  5/630 [..............................] - ETA: 29stest
test
  7/630 [..............................] - ETA: 27stest
test
  9/630 [..............................] - ETA: 26stest
test
 11/630 [..............................] - ETA: 25stest
test
 13/630 [..............................] - ETA: 25stest
test
 15/630 [..............................] - ETA: 25stest
test
 17/630 [..............................] - ETA: 24stest
test
 19/630 [..............................] - ETA: 24stest
test
 21/630 [>.............................] - ETA: 24stest
test
 23/630 [>.............................] - ETA: 24stest
test
 25/630 [>.............................] - ETA: 24stest
test
 27/630 [>.............................] - ETA: 23stest
test
 29/630 [>.............................] - ETA: 23stest
test
 31/630 [>.............................] - ETA: 23stest
test
 33/630 

415/630 [==================>...........] - ETA: 9stest
test
417/630 [==================>...........] - ETA: 9stest
test
419/630 [==================>...........] - ETA: 9stest
test
421/630 [===================>..........] - ETA: 9stest
test
423/630 [===================>..........] - ETA: 9stest
test
425/630 [===================>..........] - ETA: 9stest
test
427/630 [===================>..........] - ETA: 9stest
test
429/630 [===================>..........] - ETA: 9stest
test
431/630 [===================>..........] - ETA: 8stest
test
433/630 [===================>..........] - ETA: 8stest
test
435/630 [===================>..........] - ETA: 8stest
test
437/630 [===================>..........] - ETA: 8stest
test
439/630 [===================>..........] - ETA: 8stest
test
441/630 [====================>.........] - ETA: 8stest
test
443/630 [====================>.........] - ETA: 8stest
test
445/630 [====================>.........] - ETA: 8stest
test
447/630 [====================>.........]

In [19]:
acc_score

0.42076774097432584

In [22]:
print(confusion_matrix(y_true,y_pred))

[[8205    3    0    5    3   36    5 1370 1353 1384    0    0]
 [ 466    0    0    0    0    0    0   92   79   96    0    0]
 [ 473    1    1    0    1    1    0   76   73   67    0    0]
 [ 472    0    0    0    0    2    0   86   76   66    0    0]
 [ 464    0    0    0    0    0    0   78   88   74    0    0]
 [ 469    0    0    0    0    5    0   80   82   83    0    0]
 [ 482    0    0    0    0    1    0   77   94   65    0    0]
 [ 464    0    0    0    1    1    0   94   61   73    0    0]
 [ 444    0    0    0    0    1    1   87   89   81    0    1]
 [ 454    0    0    1    0    5    1   81   71   78    0    0]
 [ 466    1    0    0    0    0    0   89   93   61    1    0]
 [ 462    0    0    1    0    1    0   84   83   72    0    0]]


In [4]:
model_elvin=training_model(model_name="elvinouyang", class_weight=False, EPOCHS=10)

W0914 19:55:35.484839 140318015551232 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0914 19:55:35.508026 140318015551232 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0914 19:55:35.526200 140318015551232 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0914 19:55:35.527233 140318015551232 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_ba

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 99, 161, 3)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 99, 161, 3)        12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 158, 24)       1176      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 79, 24)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 79, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 77, 48)        10416     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 46, 78, 48)        0         
__________

In [5]:
y_pred_proba,y_pred, y_true, acc_score=predictions(model_elvin)

chum
20137
test
test
test
test
test
test
test
test
  1/630 [..............................] - ETA: 3:30test
test
  7/630 [..............................] - ETA: 40s test
test
 13/630 [..............................] - ETA: 27stest
test
 16/630 [..............................] - ETA: 25stest
test
 18/630 [..............................] - ETA: 25stest
test
 20/630 [..............................] - ETA: 24stest
test
 22/630 [>.............................] - ETA: 24stest
test
 24/630 [>.............................] - ETA: 24stest
test
 26/630 [>.............................] - ETA: 24stest
test
 28/630 [>.............................] - ETA: 23stest
test
 30/630 [>.............................] - ETA: 23stest
test
 32/630 [>.............................] - ETA: 23stest
test
 34/630 [>.............................] - ETA: 23stest
test
 36/630 [>.............................] - ETA: 23stest
test
 38/630 [>.............................] - ETA: 23stest
test
 40/630 [>......................

263/630 [===========>..................] - ETA: 14stest
test
265/630 [===========>..................] - ETA: 14stest
test
267/630 [===========>..................] - ETA: 14stest
test
269/630 [===========>..................] - ETA: 14stest
test
271/630 [===========>..................] - ETA: 14stest
test
273/630 [============>.................] - ETA: 14stest
test
275/630 [============>.................] - ETA: 14stest
test
277/630 [============>.................] - ETA: 14stest
test
279/630 [============>.................] - ETA: 13stest
test
281/630 [============>.................] - ETA: 13stest
test
283/630 [============>.................] - ETA: 13stest
test
285/630 [============>.................] - ETA: 13stest
test
287/630 [============>.................] - ETA: 13stest
test
289/630 [============>.................] - ETA: 13stest
test
291/630 [============>.................] - ETA: 13stest
test
293/630 [============>.................] - ETA: 13stest
test
302/630 [=============>.

513/630 [=======================>......] - ETA: 4stest
test
515/630 [=======================>......] - ETA: 4stest
test
517/630 [=======================>......] - ETA: 4stest
test
519/630 [=======================>......] - ETA: 4stest
test
521/630 [=======================>......] - ETA: 4stest
test
523/630 [=======================>......] - ETA: 4stest
test
525/630 [========================>.....] - ETA: 4stest
test
527/630 [========================>.....] - ETA: 4stest
test
529/630 [========================>.....] - ETA: 4stest
test
531/630 [========================>.....] - ETA: 4stest
test
533/630 [========================>.....] - ETA: 3stest
test
535/630 [========================>.....] - ETA: 3stest
test
537/630 [========================>.....] - ETA: 3stest
test
539/630 [========================>.....] - ETA: 3stest
test
541/630 [========================>.....] - ETA: 3stest
test
543/630 [========================>.....] - ETA: 3stest
test
545/630 [========================>.....]

In [6]:
print(confusion_matrix(y_true,y_pred))

[[8050    0    1    1    0    3    9 1409 1395 1495    1    0]
 [ 454    0    0    0    0    0    1   93   82  103    0    0]
 [ 464    0    0    0    0    0    1   76   80   72    0    0]
 [ 462    0    0    0    0    0    2   88   79   71    0    0]
 [ 452    0    0    0    0    0    0   80   93   79    0    0]
 [ 465    0    0    0    0    0    1   80   85   88    0    0]
 [ 474    0    0    0    0    0    0   79   96   70    0    0]
 [ 455    0    0    0    0    0    0   94   67   78    0    0]
 [ 439    0    0    0    0    0    2   86   90   87    0    0]
 [ 445    0    0    0    0    0    1   84   75   86    0    0]
 [ 451    0    0    0    0    0    2   90  100   68    0    0]
 [ 455    0    0    0    0    0    1   84   88   75    0    0]]


In [11]:
model_elvin_cw=training_model(model_name="elvinouyang", imbalanced=True, EPOCHS=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 99, 161, 3)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 99, 161, 3)        12        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 96, 158, 24)       1176      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 48, 79, 24)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 48, 79, 24)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 46, 77, 48)        10416     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 46, 78, 48)        0         
__________

In [12]:
y_pred_proba,y_pred, y_true, acc_score=predictions(model_elvin_cw)

chum
20137
test
test
test
test
test
test
test
test
test
 13/630 [..............................] - ETA: 34stest
test
 15/630 [..............................] - ETA: 32stest
test
 17/630 [..............................] - ETA: 31stest
test
 19/630 [..............................] - ETA: 30stest
test
 21/630 [>.............................] - ETA: 29stest
test
 23/630 [>.............................] - ETA: 29stest
test
 25/630 [>.............................] - ETA: 28stest
test
 27/630 [>.............................] - ETA: 28stest
test
 29/630 [>.............................] - ETA: 27stest
test
 31/630 [>.............................] - ETA: 27stest
test
 33/630 [>.............................] - ETA: 27stest
test
 35/630 [>.............................] - ETA: 26stest
test
 37/630 [>.............................] - ETA: 26stest
test
 39/630 [>.............................] - ETA: 26stest
test
 41/630 [>.............................] - ETA: 26stest
test
 43/630 [=>..................

test
260/630 [===========>..................] - ETA: 15stest
test
262/630 [===========>..................] - ETA: 15stest
test
264/630 [===========>..................] - ETA: 15stest
test
266/630 [===========>..................] - ETA: 15stest
test
268/630 [===========>..................] - ETA: 15stest
test
270/630 [===========>..................] - ETA: 15stest
test
272/630 [===========>..................] - ETA: 15stest
test
274/630 [============>.................] - ETA: 15stest
test
276/630 [============>.................] - ETA: 14stest
test
278/630 [============>.................] - ETA: 14stest
test
281/630 [============>.................] - ETA: 14stest
test
283/630 [============>.................] - ETA: 14stest
test
285/630 [============>.................] - ETA: 14stest
test
287/630 [============>.................] - ETA: 14stest
test
289/630 [============>.................] - ETA: 14stest
test
291/630 [============>.................] - ETA: 14stest
test
293/630 [==========

517/630 [=======================>......] - ETA: 4stest
test
519/630 [=======================>......] - ETA: 4stest
test
521/630 [=======================>......] - ETA: 4stest
test
523/630 [=======================>......] - ETA: 4stest
test
525/630 [========================>.....] - ETA: 4stest
test
527/630 [========================>.....] - ETA: 4stest
test
529/630 [========================>.....] - ETA: 4stest
test
531/630 [========================>.....] - ETA: 4stest
test
533/630 [========================>.....] - ETA: 4stest
test
535/630 [========================>.....] - ETA: 3stest
test
537/630 [========================>.....] - ETA: 3stest
test
539/630 [========================>.....] - ETA: 3stest
test
541/630 [========================>.....] - ETA: 3stest
test
543/630 [========================>.....] - ETA: 3stest
test
545/630 [========================>.....] - ETA: 3stest
test
547/630 [=========================>....] - ETA: 3stest
test
549/630 [=========================>....]

In [13]:
print(confusion_matrix(y_true,y_pred))

[[7330    1    1    0    0   58  102 1559 1586 1727    0    0]
 [ 414    0    0    0    0    4    9  100   96  110    0    0]
 [ 419    0    0    0    0    4    9   81   88   92    0    0]
 [ 422    0    0    0    0    2    6   96   94   82    0    0]
 [ 408    0    0    0    0    1    6   94   99   96    0    0]
 [ 414    0    1    0    0    4    9   88   95  108    0    0]
 [ 422    0    0    0    0    3   11   92  105   86    0    0]
 [ 419    0    0    0    0    4    7  102   75   87    0    0]
 [ 386    0    0    0    0    5   11   97  109   96    0    0]
 [ 381    0    0    0    0    3    6  100   88  113    0    0]
 [ 411    0    0    0    0    3    9   98  110   79    0    1]
 [ 422    0    0    0    0    3    3   94   96   85    0    0]]


In [15]:
acc_score

0.38084123752296767

In [21]:
model_elvin_cw_sgd=training_model(model_name="elvinouyang", optimizer="SGD", imbalanced=True, EPOCHS=30)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 99, 161, 3)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 99, 161, 3)        12        
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 96, 158, 24)       1176      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 48, 79, 24)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 48, 79, 24)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 46, 77, 48)        10416     
_________________________________________________________________
zero_padding2d_6 (ZeroPaddin (None, 46, 78, 48)        0         
__________

In [ ]:
y_pred_proba,y_pred, y_true, acc_score=predictions(model_elvin_cw_sgd)